In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import multiprocessing as mul
from scipy import stats
import pickle
from scipy.integrate import quad
from corner import corner
import pandas as pd
from scipy.stats import gaussian_kde
import dynesty as dyn

In [2]:
GRBs = ['GRB210619B', 'GRB210610B', 'GRB210204A', 'GRB201216C', 'GRB200829A', 'GRB200613A', 'GRB190114C', 'GRB180720B', 'GRB180703A', 'GRB171010A', 'GRB160625B', 'GRB160509A', 'GRB150821A', 'GRB150514A', 'GRB150403A', 'GRB150314A', 'GRB141028A', 'GRB140508A', 'GRB140206A', 'GRB131231A', 'GRB131108A', 'GRB130925A', 'GRB130518A','GRB130427A', 'GRB120119A', 'GRB100728A', 'GRB091003A', 'GRB090926A', 'GRB090618', 'GRB090328', 'GRB081221', 'GRB080916C']

In [3]:
arr = [i.replace('_results.txt', '') for i in os.listdir('./outputs/BF_xerr/') if i.endswith('.txt')]

In [4]:
arr.sort( reverse=True)
GRBs.sort(reverse=True)

In [5]:
for i in range(len(GRBs)):
    if GRBs[i] not in arr:
        print(GRBs[i])

GRB210619B
GRB180720B
GRB180703A
GRB160625B


In [2]:
all_BE = {}
for file in os.listdir('./outputs/BF_xerr/'):
    be = []
    with open('./outputs/BF_xerr/'+file, 'rb') as f:
        data = np.loadtxt(f, delimiter=',')
        # be = np.exp(data[:,0] - data[:,0][0])
        be = data[:,0] - data[:,0][0]
        # print(be)
    f.close()
    all_BE[file.replace('_results.txt', '')] = be
    # with open('./outputs/BE/'+file, 'wb') as f:
    #     np.savetxt(f, be, delimiter=',')
        
    f.close()

In [3]:
all_BE_df = pd.DataFrame(all_BE).T

In [4]:
all_BE_df

,0,1,2
GRB131231A,0.0,-1.970791,-2.873996
GRB200829A,0.0,-0.475401,-0.856664


In [5]:
all_BE_df.sort_index(inplace=True, ascending=False)

In [6]:
all_BE_df

,0,1,2
GRB200829A,0.0,-0.475401,-0.856664
GRB131231A,0.0,-1.970791,-2.873996


In [7]:
grbparam = pd.read_csv(os.getcwd() + '/../data/GRBPARAM.csv', index_col=0).T

In [8]:
grbparam.sort_index(inplace=True, ascending=False)

In [9]:
grbparam

GRB,E0,Emax,redshift
GRB210619B,10.0,11000.0,1.9370
GRB210610B,30.0,380.0,1.1300
GRB210204A,10.0,400.0,0.8760
GRB201216C,15.0,700.0,1.1000
GRB200829A,25.0,3500.0,1.2500
GRB200613A,30.0,300.0,1.2200
GRB190114C,10.0,5000.0,0.4250
GRB180720B,25.0,17000.0,0.6540
GRB180703A,20.0,400.0,0.6678
GRB171010A,10.0,620.0,0.3285


In [10]:
new_df = pd.concat([grbparam, all_BE_df], axis=1)

In [11]:
new_df.drop(['Emax'], axis=1, inplace=True)

In [12]:
new_df

,E0,redshift,0,1,2
GRB210619B,10.0,1.9370,0.0,-4.456011,0.021579
GRB210610B,30.0,1.1300,0.0,-0.850885,-0.078673
GRB210204A,10.0,0.8760,0.0,-1.697112,-0.996176
GRB201216C,15.0,1.1000,0.0,-1.358461,-0.702886
GRB200829A,25.0,1.2500,0.0,0.774136,-3.018275
GRB200613A,30.0,1.2200,0.0,-1.219409,-0.618993
GRB190114C,10.0,0.4250,0.0,9.910931,3.653995
GRB180720B,25.0,0.6540,0.0,-1.421252,-0.642453
GRB180703A,20.0,0.6678,0.0,-0.497914,-0.469741
GRB171010A,10.0,0.3285,0.0,-0.958159,-0.196392


In [13]:
new_df.sort_index(inplace=True, ascending=False)

In [14]:
new_df

,E0,redshift,0,1,2
GRB210619B,10.0,1.9370,0.0,-4.456011,0.021579
GRB210610B,30.0,1.1300,0.0,-0.850885,-0.078673
GRB210204A,10.0,0.8760,0.0,-1.697112,-0.996176
GRB201216C,15.0,1.1000,0.0,-1.358461,-0.702886
GRB200829A,25.0,1.2500,0.0,0.774136,-3.018275
GRB200613A,30.0,1.2200,0.0,-1.219409,-0.618993
GRB190114C,10.0,0.4250,0.0,9.910931,3.653995
GRB180720B,25.0,0.6540,0.0,-1.421252,-0.642453
GRB180703A,20.0,0.6678,0.0,-0.497914,-0.469741
GRB171010A,10.0,0.3285,0.0,-0.958159,-0.196392


In [15]:
data = []
for i in os.listdir('./outputs/GOF/'):
    # with open('./outputs/GOF/' + i, 'rb') as f:
    #     data.append(np.load(f))
    if i.endswith('_GOF.txt'):
        data.append([i.replace('_GOF.txt', ''), *np.loadtxt('./outputs/GOF/' + i)])
df2 = pd.DataFrame(data, columns=['GRB', 'Null', 'Linear', 'Quadratic'])
df2.index = df2['GRB']
df2.drop('GRB', axis=1, inplace=True)
df2.sort_index(inplace=True, ascending=False)

In [16]:
final = pd.concat([new_df, df2], axis=1)

In [17]:
final

,E0,redshift,0,1,2,Null,Linear,Quadratic
GRB210619B,10.0,1.9370,0.0,-4.456011,0.021579,4.297261,4.269306,4.321789
GRB210610B,30.0,1.1300,0.0,-0.850885,-0.078673,0.882673,1.357677,1.087799
GRB210204A,10.0,0.8760,0.0,-1.697112,-0.996176,8.452869,9.078137,8.992517
GRB201216C,15.0,1.1000,0.0,-1.358461,-0.702886,1.289801,1.185785,1.721224
GRB200829A,25.0,1.2500,0.0,0.774136,-3.018275,6.031649,6.228537,7.667825
GRB200613A,30.0,1.2200,0.0,-1.219409,-0.618993,0.775807,0.999109,1.047264
GRB190114C,10.0,0.4250,0.0,9.910931,3.653995,5.246489,2.769759,4.053191
GRB180720B,25.0,0.6540,0.0,-1.421252,-0.642453,1.150901,1.078413,1.742636
GRB180703A,20.0,0.6678,0.0,-0.497914,-0.469741,10.301515,11.127919,10.873344
GRB171010A,10.0,0.3285,0.0,-0.958159,-0.196392,0.616731,0.980482,0.689132


In [18]:
final = final[['E0', 'redshift', 0, 'Null', 1, 'Linear', 2, 'Quadratic']]

In [19]:
final

,E0,redshift,0,Null,1,Linear,2,Quadratic
GRB210619B,10.0,1.9370,0.0,4.297261,-4.456011,4.269306,0.021579,4.321789
GRB210610B,30.0,1.1300,0.0,0.882673,-0.850885,1.357677,-0.078673,1.087799
GRB210204A,10.0,0.8760,0.0,8.452869,-1.697112,9.078137,-0.996176,8.992517
GRB201216C,15.0,1.1000,0.0,1.289801,-1.358461,1.185785,-0.702886,1.721224
GRB200829A,25.0,1.2500,0.0,6.031649,0.774136,6.228537,-3.018275,7.667825
GRB200613A,30.0,1.2200,0.0,0.775807,-1.219409,0.999109,-0.618993,1.047264
GRB190114C,10.0,0.4250,0.0,5.246489,9.910931,2.769759,3.653995,4.053191
GRB180720B,25.0,0.6540,0.0,1.150901,-1.421252,1.078413,-0.642453,1.742636
GRB180703A,20.0,0.6678,0.0,10.301515,-0.497914,11.127919,-0.469741,10.873344
GRB171010A,10.0,0.3285,0.0,0.616731,-0.958159,0.980482,-0.196392,0.689132


In [20]:
final['Null'] = final['Null'].round(1)
final['Linear'] = final['Linear'].round(1)
final['Quadratic'] = final['Quadratic'].round(1)
final[1] = final[1].round(1)
final[2] = final[2].round(1)

In [21]:
final

,E0,redshift,0,Null,1,Linear,2,Quadratic
GRB210619B,10.0,1.9370,0.0,4.3,-4.5,4.3,0.0,4.3
GRB210610B,30.0,1.1300,0.0,0.9,-0.9,1.4,-0.1,1.1
GRB210204A,10.0,0.8760,0.0,8.5,-1.7,9.1,-1.0,9.0
GRB201216C,15.0,1.1000,0.0,1.3,-1.4,1.2,-0.7,1.7
GRB200829A,25.0,1.2500,0.0,6.0,0.8,6.2,-3.0,7.7
GRB200613A,30.0,1.2200,0.0,0.8,-1.2,1.0,-0.6,1.0
GRB190114C,10.0,0.4250,0.0,5.2,9.9,2.8,3.7,4.1
GRB180720B,25.0,0.6540,0.0,1.2,-1.4,1.1,-0.6,1.7
GRB180703A,20.0,0.6678,0.0,10.3,-0.5,11.1,-0.5,10.9
GRB171010A,10.0,0.3285,0.0,0.6,-1.0,1.0,-0.2,0.7


In [22]:
final.to_latex("./outputs/table.tex", float_format="%.2f")

In [31]:
temp = final.sort_values(1, ascending=False)

temp

,E0,redshift,0,Null,1,Linear,2,Quadratic
GRB190114C,10.0,0.4250,0.0,5.2,9.9,2.8,3.7,4.1
GRB131231A,10.0,0.6420,0.0,4.9,6.7,4.3,5.8,3.6
GRB130925A,10.0,0.3470,0.0,4.5,5.4,3.7,4.2,3.8
GRB200829A,25.0,1.2500,0.0,6.0,0.8,6.2,-3.0,7.7
GRB091003A,10.0,0.8969,0.0,3.0,-0.1,3.0,0.0,3.3
GRB150821A,10.0,0.7550,0.0,0.8,-0.4,1.1,0.3,0.9
GRB180703A,20.0,0.6678,0.0,10.3,-0.5,11.1,-0.5,10.9
GRB090328,30.0,0.7360,0.0,7.3,-0.6,8.0,-0.2,7.4
GRB090618,10.0,0.5400,0.0,0.5,-0.7,0.8,-0.4,0.4
GRB100728A,40.0,1.5670,0.0,2.6,-0.8,2.6,-0.1,2.6


In [35]:
np.log(100)

4.605170185988092

In [32]:
temp[temp[1] > np.log(100)]

,E0,redshift,0,Null,1,Linear,2,Quadratic
GRB190114C,10.0,0.425,0.0,5.2,9.9,2.8,3.7,4.1
GRB131231A,10.0,0.642,0.0,4.9,6.7,4.3,5.8,3.6
GRB130925A,10.0,0.347,0.0,4.5,5.4,3.7,4.2,3.8


In [34]:
temp[temp[2] > np.log(100)]

,E0,redshift,0,Null,1,Linear,2,Quadratic
GRB131231A,10.0,0.642,0.0,4.9,6.7,4.3,5.8,3.6
